# REWOO Pattern Airline Assistant

## Overview
In this example we will guide you through how to create a REWOO (Reasoning Without Observation) pattern implementation using Strands multiagent orchestration. We will demonstrate a three-agent system that separates reasoning (planning) from acting (tool execution), enabling more reliable and debuggable agent workflows for airline customer service tasks.

## Agent Details
<div style="float: left; margin-right: 20px;">
    
|Feature             |Description                                        |
|--------------------|---------------------------------------------------|
|Native tools used   |generate_flight_plan, execute_flight_plan, solve_flight_query|
|Custom tools created|All three core REWOO tools with airline domain integration|
|Agent Structure     |Three-agent sequential pipeline (Planner → Worker → Solver)|
|AWS services used   |Amazon Bedrock (Claude 3.7 Sonnet)                |
|Domain tools        |MAbench airline tools (booking, search, updates)  |

</div>


## Architecture
<div style="text-align:center">
    <img src="../images/rewoo.png" alt="REWOO Architecture" width="600">
    <p>The system consists of three specialized agents connected in a sequential graph:</p>
    <p><em>REWOO Architecture: User Query → [Planner] → [Worker] → [Solver] → Final Response</em></p>
</div>
## Key Features
* **Three-agent sequential pipeline**: Planner generates structured plans, Worker executes tools, Solver synthesizes responses
* **Separation of concerns**: Clear distinction between reasoning (planning) and acting (execution)
* **Structured execution**: Plans use #E1, #E2, #E3 format for deterministic tool execution
* **REPEAT block support**: Handles iterative operations for batch processing
* **Evidence-based responses**: Final answers grounded in actual tool execution results
* **Airline domain integration**: Complete integration with MAbench airline tools and TauBench datasets

In [ ]:
!pip3 install -r ./requirements.txt --quiet 
!pip3 install strands-agents==v1.6.0 strands-agents-tools==v0.2.5 --quiet
!pip3 install litellm --quiet


## Importing dependency packages

Now let's import all the necessary libraries and modules for our REWOO implementation. This includes standard Python libraries, AWS SDK components, Strands framework modules, and custom helper functions.

In [ ]:
import time, os, re, json, ast, logging
from typing import List, Dict, Any
from strands import Agent, tool
from strands.models import BedrockModel
from strands.multiagent.graph import GraphBuilder
from strands.agent import AgentResult
from strands.types.content import Message
from strands.telemetry.metrics import EventLoopMetrics
from helpers.rewoo_helper_funcs import *
from helpers.bedrock_helper import get_bedrock_response, get_claude_response, get_claude_response_text

## Configure Strands Framework

Now let's set up the core Strands framework components that will power our REWOO multiagent system. We need to configure the AWS Bedrock connection, conversation management, and logging to ensure our three-agent pipeline runs smoothly.

### Framework Setup Process

First, we'll establish the **AWS region** and create a `BedrockModel` instance that all three REWOO agents (Planner, Worker, Solver) will share. We do this so that all agents use the same LLM configuration for consistent behavior. 

Finally, we'll configure **logging** to minimize noise during execution so we can focus on the REWOO execution flow and results.

In [ ]:
# Create BedrockModel with specified region
region="us-east-1"
bedrock_model_taubench = BedrockModel(region_name= region)

#setup logging
# Disable all logging except critical errors
logging.basicConfig(level=logging.CRITICAL)

# Silence specific noisy loggers completely
for logger_name in ["strands", "graph", "event_loop", "registry", "sliding_window_conversation_manager", "bedrock", "streaming"]:
    logging.getLogger(logger_name).setLevel(logging.CRITICAL)


## Import airline domain tools

Now we'll import the comprehensive set of airline domain tools from MAbench and TauBench. These tools provide the actual functionality that our REWOO Worker agent will execute, including flight booking, reservation management, and customer service operations.

In [ ]:
# Libraries

import sys
sys.path.append('../data/ma-bench/')
sys.path.append('../data/tau-bench/')

from mabench.environments.airline.tools.book_reservation import book_reservation
from mabench.environments.airline.tools.calculate import calculate
from mabench.environments.airline.tools.cancel_reservation import cancel_reservation
from mabench.environments.airline.tools.get_reservation_details import get_reservation_details
from mabench.environments.airline.tools.get_user_details import get_user_details
from mabench.environments.airline.tools.list_all_airports import list_all_airports
from mabench.environments.airline.tools.search_direct_flight import search_direct_flight
from mabench.environments.airline.tools.search_onestop_flight import search_onestop_flight
from mabench.environments.airline.tools.send_certificate import send_certificate
from mabench.environments.airline.tools.think import think
from mabench.environments.airline.tools.transfer_to_human_agents import transfer_to_human_agents
from mabench.environments.airline.tools.update_reservation_baggages import update_reservation_baggages
from mabench.environments.airline.tools.update_reservation_flights import update_reservation_flights
from mabench.environments.airline.tools.update_reservation_passengers import update_reservation_passengers

domain = "airline"

from tau_bench.envs.airline.wiki import WIKI

## REWOO Orchestration

ReWOO reframes *"how tools are used"* rather than *"which tools exist."* We keep a single tool-executor for all airline APIs, but we enforce a **plan → execute → synthesize** separation around it. In Strands, this becomes a small, explicit graph where each node returns a typed result (`AgentResult`) and the runtime forwards those results downstream in a deterministic way. This leads to **governance**, **observability**, and **repeatability**.

### Create PLANNER: Receives user query and makes the plan

Now let's create the first component of our REWOO system - the **Planner tool**. This is where the *"Reasoning without Observation"* happens. The Planner's job is to create structured execution plans without actually running any tools.

#### Planner specific tool

We first define a `generate_flight_plan` tool that takes a user query and converts it into a step-by-step plan using the **#E1, #E2, #E3** format. The tool includes a comprehensive prompt that:

- Lists all available airline tools
- Provides detailed examples of how to structure plans for different scenarios like flight changes, new bookings, and passenger updates
- Supports **REPEAT blocks** for handling batch operations like processing multiple reservations

We create a specialized planning agent that uses this prompt along with airline policy knowledge from the **WIKI** to generate reliable, structured and policy compliant plans that the Worker agent can execute deterministically.


In [ ]:
@tool
def generate_flight_plan(user_query: str) -> str:
    """Generate a structured flight plan for the given user query"""
    print(f"inside flight plan tool \n")
    # planner prompt
    planner_prompt = """
# PLANNING ONLY ASSISTANT - DO NOT EXECUTE

Your ONLY job is to write a plan using the exact format below. You MUST NOT try to execute the plan or have any other interactions.

## Available Flight Tools
* calculate[expression]
* get_reservation_details[reservation_id]
* update_reservation_flights[reservation_id, cabin, flights, payment_id]
* search_onestop_flight[origin, destination, date]
* send_certificate[user_id, amount]
* cancel_reservation[reservation_id]
* search_direct_flight[origin, destination, date]
* get_user_details[user_id]
* list_all_airports[]
* book_reservation[user_id, origin, destination, flight_type, cabin, flights, passengers, payment_methods, total_baggages, nonfree_baggages, insurance]
* think[thought]
* transfer_to_human_agents[summary]
* update_reservation_passengers[reservation_id, passengers]
* update_reservation_baggages[reservation_id, total_baggages, nonfree_baggages, payment_id]
* book_reservation[user_id, origin, destination, flight_type, cabin, flights, passengers, payment_methods, total_baggages, nonfree_baggages, insurance]
* cancel_reservation[reservation_id]
* calculate[expression]

## REPEAT Syntax and Usage
When multiple iterations of the same steps are needed, use this format:

1. First, use think tool to analyze and count items to process
2. Then, use another think tool to plan iteration details
3. Finally, use REPEAT block with the count from previous steps

REPEAT(count_from_previous_step) {
    tool1[parameters]
    tool2[parameters]
    ...
}

Available variables in REPEAT blocks:
- CURRENT_ITERATION (0-based index)
- CURRENT_ITEM (from list being processed)
- Other variables extracted from previous steps

Use REPEAT blocks when:
- Processing multiple reservations
- Applying multiple certificates
- Handling multiple passengers
- Any task that requires the same steps multiple times

Note: Evidence numbers inside REPEAT will be expanded sequentially


## Required Format - USE EXACTLY THIS:

Plan 1: [Description]
#E1 = [tool_name][parameters]

Plan 2: [Description]
#E2 = [tool_name][parameters]

## Examples:


Example 1 : "Can you put me on an earlier flight? My reservation ID is 'CD789012'"
Plan 1: Retrieve the current reservation details
#E1 = get_reservation_details[reservation_id="CD789012"]

Plan 2: Search for earlier direct flights based on the origin, destination and date from #E1
#E2 = search_direct_flight[origin=origin_airport_code, destination=destination_airport_code, date=travel_date]

Plan 3: Update the reservation with the earlier flight found in #E2 and use details from #E1 and useer query as necessary
#E3 = update_reservation_flights[reservation_id="CD789012", cabin=cabin_class, flights=selected_flights, payment_id=payment_info]

Example 2 : "My user id is mia_li_3668. I want to fly from New York to Seattle on May 20 (one way). I do not want to fly before 11am EST. I want to fly in economy. I prefer direct flights but one stopover is also fine. If there are multiple options, I prefer the one with the lowest price. I have 3 baggages. I do not want insurance. I want to use my two certificates to pay. If only one certificate can be used, I prefer using the larger one, and pay the rest with my 7447 card"
Plan 1: Get user details to check available certificates
#E1 = get_user_details[user_id="mia_li_3668"]

Plan 2: Get list of airports to find the airport codes for New York and Seattle
#E2 = list_all_airports[]

Plan 3: Search for direct flights using airport codes from #E2 and date from given user question
#E3 = search_direct_flight[origin=origin_airport_code, destination=destination_airport_code, date=travel_date]

Plan 4: If no suitable direct flights, search for one-stop flights using using airport codes from #E2 and date from given user question
#E4 = search_onestop_flight[origin=origin_airport_code, destination=destination_airport_code, date=travel_date]

Plan 5: Return selected flights from #E4  and #E3 


Example 3 : "I have a booking number TR7845. I need to update my daughter's name from Emma Wilson to Emma Thompson as she recently got married. I'm Jennifer Wilson, ID: TW5432P891."

Plan 1: Retrieve the current reservation details
#E1 = get_reservation_details[reservation_id="TR7845"]

Plan 2: Verify user identity and authorization
#E2 = get_user_details[user_id="TW5432P891"]

Plan 3: Think about the passenger name changes needed
#E3 = think["Analyze the user query and reservation details:

Identify the passenger whose name needs to be changed: Emma Wilson
New name for this passenger: Emma Thompson
Keep all other passengers unchanged
Preserve existing passenger details (DOB, etc) from reservation
Create an updated passenger list with the name change"]

Plan 4: Update the reservation with modified passenger information
#E4 = update_reservation_passengers[reservation_id="TR7845", passengers=[
{"first_name": "Jennifer", "last_name": "Wilson", "dob": extracted_dob_jennifer},
{"first_name": "Emma", "last_name": "Thompson", "dob": extracted_dob_emma}
]]

Example 4 : "Hi, my name is Jordan Smith (customer ID: ZX7890Y123). I have a reservation with booking code LM5678 for a flight from Chicago to Miami on June 15. 
I need to add my son, Alex Smith, to the reservation and include an extra bag for him. Can you help me with that?"

Plan 1: Retrieve the current reservation details
#E1 = get_reservation_details[reservation_id="LM5678"]

Plan 2: Verify user identity and authorization
#E2 = get_user_details[user_id="ZX7890Y123"]

Plan 3: Think about required passenger updates
#E3 = think["Analyze current reservation and requested changes:

Get existing passenger list from #E1
New passenger to add: Alex Smith (son)
Need to preserve all existing passenger details
Create updated passenger list that includes both existing and new passengers"]

Plan 4: Update the reservation with complete passenger list
#E4 = update_reservation_passengers[reservation_id="LM5678", passengers=[
extract_existing_passengers_from_E1,
{"first_name": "Alex", "last_name": "Smith", "type": "child"}
]]

Plan 5: Think about baggage update
#E5 = think["Calculate baggage updates:

Get current total_baggages from #E1
Add one extra bag for new passenger
Determine if extra bag is free or paid based on cabin class"]

Plan 6: Update the baggage count
#E6 = update_reservation_baggages[
reservation_id="LM5678",
total_baggages=current_total_plus_one,
nonfree_baggages=current_nonfree_plus_one,
payment_id=payment_from_context
]

Example 5: "My user id is ABC123. I want to downgrade all my business flights to economy for my reservations. Please calculate total savings."

Plan 1: Get user details to retrieve all reservations
#E1 = get_user_details[user_id="ABC123"]

Plan 2: REPEAT(length_of_reservations_from_#E1) {
    get_reservation_details[reservation_id=CURRENT_RESERVATION_ID]    
    calculate["current_savings = business_fare - economy_fare"]
    calculate["total_savings += current_savings"]
    update_reservation_flights[reservation_id=CURRENT_RESERVATION_ID, cabin="economy", flights=CURRENT_FLIGHTS, payment_id=CURRENT_PAYMENT]
}


## IMPORTANT: 
1. ONLY write the plan - nothing else
2. Do NOT add any explanations or clarifications
3. Do NOT attempt to execute any actions
4. Follow the format exactly as shown
5. Use 'think' tool only when needed like name change.


<policy>
{policy}
</policy>
"""

    planning_llm = Agent(
        model=bedrock_model_taubench,
        system_prompt=planner_prompt.replace("{policy}", WIKI)
    )
    plan = planning_llm(user_query)
    
    return str(plan)



## Define direct LLM call utility

We also need a utility function for direct LLM calls. This is used for specific REWOO operations like **parameter resolution** and **plan generation** where we need deterministic, focused responses.


In [ ]:
def direct_llm_call(prompt):
    max_tokens = 2048
    temp = 0
    topP = 1
    response = get_claude_response(user_message=prompt,
                                    model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0", 
                                    max_tokens=max_tokens, 
                                   temp=temp)                
    answer = get_claude_response_text(response)
    return answer


## Create EXECUTOR: receives the plan and executes it

Now let's build the second component of our REWOO system - the **Worker tool**. This is where the actual *"Observation"* happens as we execute the structured plan from the Planner and collect evidence from each step.

### Plan Execution Process

The `execute_flight_plan` tool takes the structured plan and parses it to identify individual steps (**#E1, #E2**, etc.) and **REPEAT blocks**. We use regular regex patterns to extract both regular steps and iterative operations. The tool then executes each step in sequence, maintaining an **evidence dictionary** that accumulates results from each tool call.

### Step Types

For **regular steps**, we call `execute_single_step` which handles parameter resolution and tool execution. For **REPEAT blocks**, we use `handle_repeat_block` to process iterative operations like updating multiple reservations. 

Each execution step builds upon previous evidence, so later steps can reference results from earlier ones. This creates a **chain of evidence** that the Solver can use to generate the final response.



In [ ]:
import json
from typing import Any, Dict, List, Optional, Union

@tool
def execute_flight_plan(plan: str) -> str:
    """Execute plan with fully dynamic parameter resolution"""

    user_query=  extract_original_task(plan)
    
    steps = []
    regular_step_pattern = r'#E(\d+)\s*=\s*(\w+)\[([^\]]*)\]'
    repeat_block_pattern = r'#E(\d+)\s*=\s*REPEAT\(([^)]+)\)\s*\{([^}]+)\}'
    
    # First find all regular steps
    regular_steps = re.finditer(regular_step_pattern, plan)
    for match in regular_steps:
        steps.append(('regular', match.group(1), match.group(2), match.group(3)))
        
    # Then find REPEAT blocks
    repeat_blocks = re.finditer(repeat_block_pattern, plan)
    for match in repeat_blocks:
        steps.append(('repeat', match.group(1), match.group(2), match.group(3)))
    
    # Sort steps by evidence number
    steps.sort(key=lambda x: int(x[1]))
    
    all_evidence = {}  # This will store ALL evidences
    current_evidence_num = 1
    
    for step_type, evidence_id, *rest in steps:
        if step_type == 'regular':
            tool_name, args_str = rest
            new_evidence = execute_single_step(evidence_id, tool_name, args_str, all_evidence.copy(), user_query)
            # Merge new evidence into all_evidence
            all_evidence.update(new_evidence)
            
        else:
            repeat_condition, block_content = rest            
            repeat_evidence = handle_repeat_block(evidence_id, repeat_condition, block_content, all_evidence.copy(), user_query)
            # Merge repeat evidence into all_evidence
            all_evidence.update(repeat_evidence)
            
    return str(all_evidence) #all_evidence


## Helper functions for executor

Now let's look at how we execute each individual step in our REWOO plan. The `execute_single_step` function is the workhorse that takes each **#E1, #E2, #E3** step and actually runs the corresponding airline tool.

### Step Execution Process

First, the function **parses the argument string** (like `"reservation_id=CD789012, cabin=economy"`) into a proper dictionary that Python can work with. Then it builds context from all previous evidence steps so that later steps can reference earlier results - this is crucial for the REWOO pattern where each step builds on previous ones.

For most tools, we use our smart `resolve_arguments` function to convert vague parameter values into the exact formats the airline APIs need. But we handle two special cases differently:
- The **"think" tool** just stores reasoning text
- The **"calculate" tool** uses an LLM to extract mathematical expressions from the context

### Argument Resolution

We use `resolve_arguments()` defined in `rewoo_helpers.py` to extract the arguments and their values accurately before calling the particular airline tool. The `resolve_arguments` function intelligently converts vague parameter values from the plan into the exact formats required by airline tools. 

It takes parameter names and their rough values (like *"New York"* for origin), then uses an LLM agent to extract the precise format needed (like *"JFK"* airport code). The function has built-in knowledge of airline domain requirements:
- **Airports** need 3-letter codes
- **Dates** need YYYY-MM-DD format  
- **Cabin classes** have specific values like "economy" or "business"
- **Complex parameters** like flight lists need proper JSON arrays

For each parameter, it creates a specialized prompt that tells the LLM exactly what format to return, then handles type conversion (strings, integers, JSON arrays) to ensure the airline tools receive properly formatted data. This bridges the gap between human-readable plan parameters and the strict data formats that airline APIs require.

Finally, we execute the actual airline tool (like getting reservation details or searching flights) and carefully store both successful results and any errors in our evidence dictionary. This evidence becomes the foundation for our Solver agent to create the final user response.




In [ ]:
def execute_single_step(evidence_id: str, tool_name: str, args_str: str, evidence: dict, user_query: str):
    """
    Execute a single tool step and gather evidence from its execution.
    
    Args:
        evidence_id (str): Unique identifier for this evidence step
        tool_name (str): Name of the tool to execute
        args_str (str): String containing comma-separated key=value argument pairs
        evidence (dict): Dictionary containing previous evidence steps
        user_query (str): Original user query for context
        
    Returns:
        dict: Evidence gathered from this step's execution
    """
    print(f"\nDEBUG: Processing step #E{evidence_id}")
    print(f"\nDEBUG: Tool name: {tool_name}")
    
    # Initialize evidence storage for this step
    step_evidence = {}
    
    # Parse string arguments into kwargs dictionary
    kwargs = {}
    if args_str.strip():
        for arg_pair in args_str.split(','):
            arg_pair = arg_pair.strip()
            if '=' in arg_pair:
                key, value = arg_pair.split('=', 1)
                key = key.strip()
                value = value.strip().strip('"\'')
                kwargs[key] = value
    
    print(f"DEBUG: Final kwargs: {kwargs}")
    
    # Build context from previous evidence
    items = list(evidence.items()) 
    evidence_context = "\n".join([f"{k}: {v['results']}" for k, v in evidence.items()])
    context_dict = items    
    context = f"Evidence Context {evidence_context}\n\nUser Query: {user_query}"
   
    # Resolve arguments using LLM for non-think tools
    if 'think' not in tool_name:           
        kwargs = resolve_arguments(tool_name, kwargs, evidence, context, bedrock_model_taubench) 
        
    print(f"New kwargs {kwargs}")
    
    # Execute the tool
    try:
        tool_func = getattr(worker_agent.tool, tool_name)
        
        if 'think' in tool_name:
            # Special handling for think tool
            result = f" "
            
        elif 'calculate' in tool_name:
            # Special handling for calculate tool
            calculate_prompt = """From the given <user_query>  and <evidence>  find the values that can be used for the <calculator_kwargs> that will be 
            passed to calculate tool. You must only return a math expression  to calculate, such as '2 + 2' which can be used by the 'calculate' tool. 
            You must only return the mathematical expression between 2 quotation marks.
            <user_query>
            {user_query}
            </user_query>
            <evidence>
            {evidence_context}
            </evidence>
            <calculator_kwargs>
            {args_str}
            </calculator_kwargs>
            """
            calculate_kwargs = direct_llm_call(system_prompt=calculate_prompt) 
            result = tool_func(calculate_kwargs)
            print(f"answer from calculate {calculate_kwargs}  {result} \n")
            
        else:
            # Standard tool execution
            if kwargs:
                print(f"DEBUG: Calling {tool_name} with kwargs: {kwargs}")
                result = tool_func(**kwargs)
            else:
                print(f"DEBUG: Calling {tool_name} with no args")
                result = tool_func()
        
        print(f"DEBUG: Tool result: {result}")
       
        # Process and store the result
        if isinstance(result, dict) and 'content' in result:
            result_data = result['content'][0]['text']
        else:
            result_data = str(result)
        
        step_evidence[f'#E{evidence_id}'] = {
            'evidence_id': f'#E{evidence_id}',
            'description': f"Execute {tool_name} with {kwargs if kwargs else 'no parameters'}",
            'results': result_data
        }
            
    except Exception as e:
        # Handle and store any errors
        step_evidence[f'#E{evidence_id}'] = {
            'evidence_id': f'#E{evidence_id}',
            'description': f"Execute {tool_name} with {kwargs if kwargs else 'no parameters'}",
            'results': f"Error: {str(e)}"
        }
    
    return step_evidence

## Handle Batch Operations with REPEAT Blocks

Now let's explore how our REWOO system handles batch operations through **REPEAT blocks**. The `handle_repeat_block` function enables us to process multiple items with the same set of operations - like updating several reservations or applying multiple certificates.

The function starts by using an LLM to analyze the previous evidence and determine how many items need processing. For example, if a user has 3 reservations that need updating, the LLM examines the evidence from earlier steps and extracts both the count (3) and the list of reservation IDs to process. We need a way for the system to automatically figure out how many times to repeat an operation based on the actual data from previous steps.

The `REPEAT_ANALYSIS_PROMPT` is a specialized template that helps our LLM analyze JSON responses from earlier evidence steps and determine iteration counts dynamically. For example, when a user says *"downgrade all my business flights to economy,"* we first get their user details which contains a list of reservations. This prompt then instructs the LLM to examine that JSON data, count how many reservations exist, and extract the specific reservation IDs to process.

We use this prompt in our `handle_repeat_block` function to make REPEAT operations truly dynamic - instead of hardcoding "repeat 3 times," the system intelligently determines "this user has 3 reservations, so repeat 3 times with these specific IDs." This makes our REWOO system capable of handling real-world scenarios where the number of items to process varies by user and situation.



In [ ]:
REPEAT_ANALYSIS_PROMPT = """Given this JSON response from a previous step:
{json_data}

And this REPEAT condition:
{repeat_condition}

Task:
1. Determine which list in the JSON needs to be counted
2. Count the number of items in that list
3. Extract all items from that list

Return ONLY in this exact format:
Count: [number]
List: [comma-separated items]"""


Once we know what to iterate over, the function loops through each item and executes the same set of tools for each one. It replaces placeholder variables like `CURRENT_RESERVATION_ID` with the actual reservation ID for that iteration, and `CURRENT_ITERATION` with the loop counter. Each iteration calls our `execute_single_step` function to run the airline tools, building up evidence as it goes.

This **REPEAT capability** is essential for airline scenarios where customers often need bulk operations - like a business traveler wanting to downgrade all their flights from business to economy, or a family needing to add baggage to multiple reservations. The function makes these complex batch operations as reliable and traceable as single operations.


In [ ]:

def handle_repeat_block(evidence_id: str, repeat_condition: str, block_content: str, evidence: dict, user_query: str):
    """Handle execution of a REPEAT block"""
    # Extract source evidence number
    print("inside handle repeat block \n")
    repeat_evidence = {}  # Store evidence from repeat block
   
    # Use LLM to analyze JSON and repeat condition
    system_prompt = REPEAT_ANALYSIS_PROMPT.format(
        json_data=evidence,
        repeat_condition=repeat_condition  # e.g., "number_of_reservations_from_#E1"
    )
    llm_response = direct_llm_call(system_prompt)    
    count_match = re.search(r'Count:\s*(\d+)', llm_response)
    list_match = re.search(r'List:\s*([\w\d, ]+)', llm_response)  # More flexible pattern

    if not count_match:
        raise ValueError(f"Could not find count in LLM response: {llm_response}")
    
    count = int(count_match.group(1))
    
    if not list_match:
        # Fallback: try to extract items between commas after "List:"
        list_start = llm_response.find("List:") + 5
        items_text = llm_response[list_start:].strip()
        items = [item.strip() for item in items_text.split(',')]
    else:
        items = [item.strip() for item in list_match.group(1).split(',')]

    # Execute block for each item
    current_evidence_num = int(evidence_id)
    for i, item in enumerate(items):
        print(f"\nDEBUG: REPEAT iteration {i+1}/{count}")
        
        # Parse and execute each step in the block
        step_pattern = r'(\w+)\[([^\]]*)\]'
        steps = re.findall(step_pattern, block_content)
        
        for tool_name, args_str in steps:
            # Replace placeholders
            print(f" tool_name {tool_name}  args_str {args_str} \n")
            processed_args = args_str.replace('CURRENT_RESERVATION_ID', item)
            processed_args = processed_args.replace('CURRENT_ITERATION', str(i))
            
            step_evidence = execute_single_step(str(current_evidence_num), tool_name, processed_args, evidence, user_query)
            repeat_evidence.update(step_evidence)
            current_evidence_num += 1
    
    return repeat_evidence

## Create SOLVER: Receives full plan and the responses of individual tool calls and prepares final response which is given to the user

Now let's build the final component of our REWOO system - the Solver tool. This is where we transform all the technical evidence collected by the Worker into a natural, user-friendly response that actually answers the customer's question.

The `solve_flight_query` tool starts by parsing the evidence string back into a structured dictionary so we can work with the individual results from each step. Then it reconstructs the execution flow by building a plan summary that shows what each step accomplished - this gives the LLM context about what actions were taken and what data was gathered.

The `solve_prompt` template is the key to this process. It presents the LLM with both the step-by-step evidence and the original user query, then asks it to synthesize everything into a direct answer. The prompt specifically warns about potentially irrelevant information in long evidence chains and instructs the LLM to respond concisely without extra explanations.

Finally, we use our direct LLM call to generate the final response. This approach ensures that our answer is grounded in actual tool execution results rather than hallucinated information - a crucial aspect of the REWOO pattern that makes responses reliable and traceable back to concrete actions taken by the airline tools.


In [ ]:
solve_prompt = """Solve the following task or problem. To solve the problem, we have made step-by-step Plan and retrieved corresponding Evidence to each Plan. Use them with caution since long evidence might contain irrelevant information.

{plan}

Now solve the question or task according to provided Evidence above. Respond with the answer directly with no extra words.

Task: {task}
Response:"""


@tool
def solve_flight_query(user_query: str, planner_response: str, evidence_str: str) -> str:
    """
    Solve user query using structured evidence from worker execution
    """
   
    # Parse evidence string to dictionary
    try:        
        evidence_dict = ast.literal_eval(evidence_str)       
    except Exception as e:
        print(f"DEBUG: Failed to parse evidence_str: {e}")
        evidence_dict = {}
    
    # Build plan string from evidence
    plan = ""
    print(f"\nDEBUG: Building plan from evidence_dict with {len(evidence_dict)} items")
    for evidence_id, evidence_data in evidence_dict.items():
        
        if isinstance(evidence_data, dict):
            description = evidence_data.get('description', '')
            results = evidence_data.get('results', '')
            plan += f"Plan: {description}\n{evidence_id} = {results}\n\n"
            
    
    print(f"DEBUG: Final plan built")
    
    try:
        
        formatted_prompt = solve_prompt.format(plan=plan, task=user_query)        
        solved_answer = direct_llm_call(formatted_prompt)       
        result = str(solved_answer)
        return result
        
    except Exception as e:
        print(f"DEBUG: Exception in solve_flight_query: {e}")
        print(f"DEBUG: Exception type: {type(e)}")
        raise

## Make the REWOO strands graph

## Create Custom Agent Classes for REWOO Pipeline

Now let's assemble our three REWOO components into a working multiagent system. Since the Strands multiagent graph requires specific communication protocols, we need to create custom agent classes that extend the base `Agent` class with specialized `stream_async` methods.

Each agent follows the same pattern: it receives input, calls its specialized tool, then packages the result in the proper `AgentResult` format that the multiagent graph expects:

- **PlannerAgent** uses only the `generate_flight_plan` tool to create structured plans
- **WorkerAgent** has access to all the airline domain tools plus the `execute_flight_plan` orchestrator - this gives it everything needed to execute any plan step  
- **SolverAgent** uses the `solve_flight_query` tool to synthesize final responses

The key technical detail is the `stream_async` method in each agent. This method must:
1. Normalize the input prompt
2. Call the appropriate tool
3. Create a proper `Message` object with the results
4. Wrap it in an `AgentResult` with metrics and state information
5. Yield it in the format the graph expects

This ensures seamless communication between agents as data flows from Planner → Worker → Solver.

Each agent instance is configured with the shared Bedrock model and its specific tools, creating a cohesive system where each component has a clear, specialized role in the REWOO pattern.



In [ ]:
# Build rewoo graph

class PlannerAgent(Agent):
    async def stream_async(self, prompt: str):
    
        # NOTE: stream_async must accept **kwargs; Graph/Swarm pass callback_handler & more.
        print(f"DEBUG: PLANNER AGENT CALLED\n")

        # Call the tool and get result        
        prompt=normalize_prompt(prompt)        
        plan_result = self.tool.generate_flight_plan(user_query=prompt)
        
        # Create  AgentResult object with required parameters
        message = Message(content=[{"text": str(plan_result)}])
        metrics = EventLoopMetrics()
        print("DEBUG: PLANNER AGENT RESULT: \n", json.dumps(message), "\n")
        agent_result = AgentResult(
            stop_reason="end_turn",
            message=message,
            metrics=metrics,
            state=None
        )
        
        # Yield the result event that multiagent graph expects
        yield {"result": agent_result}

# Use Custom planner
planner_agent = PlannerAgent(
    model=bedrock_model_taubench,
    tools=[generate_flight_plan],
    name="planner"
)

class WorkerAgent(Agent):
    async def stream_async(self, prompt: str):
        # Call the execute_flight_plan tool with the plan from planner
        print(f"\n DEBUG: WORKER AGENT CALLED TO EXECUTE PLAN \n")
        prompt=normalize_prompt(prompt)
        
        evidence_result = self.tool.execute_flight_plan(plan=prompt)#add user query argument
        
        # Create AgentResult object
        message = Message(content=[{"text": str(evidence_result)}])
        print("DEBUG: WORKER AGENT RESULT: \n", json.dumps(message), "\n")
        metrics = EventLoopMetrics()
        
        agent_result = AgentResult(
            stop_reason="end_turn",
            message=message,
            metrics=metrics,
            state=None
        )
        
        # Yield the result event that multiagent graph expects
        yield {"result": agent_result}

# Use the custom worker
worker_agent = WorkerAgent(
    model=bedrock_model_taubench,
    tools=[
        book_reservation,
        calculate,
        cancel_reservation,
        get_reservation_details,
        get_user_details,
        list_all_airports,
        search_direct_flight,
        search_onestop_flight,
        send_certificate,
        think,
        transfer_to_human_agents,
        update_reservation_baggages,
        update_reservation_flights,
        update_reservation_passengers,
        execute_flight_plan
    ],
    name="worker"
)

class SolverAgent(Agent):
    async def stream_async(self, prompt: str):
        # Extract plan and evidence from the combined input
        # The prompt will contain both original task and worker results
        print(f"DEBUG: SOLVER AGENT CALLED TO FORM FINAL ANSWER FROM EXECUTED PLAN\n")
        prompt=normalize_prompt(prompt)
        
        original_task, evidence_str = extract_task_and_plans(prompt)
        print(f"ORIGINAL_TASK {original_task}\n")
        print(f"EVIDENCE STRING {evidence_str}\n")
        # Call solve_flight_query tool
        final_answer = self.tool.solve_flight_query(
            user_query=original_task,
            planner_response="",  # Not needed for solver
            evidence_str=str(evidence_str)
        )
        
        # Create AgentResult object
        message = Message(content=[{"text": str(final_answer)}])
        print("DEBUG: SOLVER AGENT RESULT: \n", json.dumps(message), "\n")
        metrics = EventLoopMetrics() # check how to get the eventloopmetrics
        
        agent_result = AgentResult(
            stop_reason="end_turn",
            message=message,
            metrics=metrics,
            state=None
        )
        
        yield {"result": agent_result}


# Use the custom solver
solver_agent = SolverAgent(
    model=bedrock_model_taubench,
    tools=[solve_flight_query],
    name="solver"
)


## Assemble the complete REWOO graph

Finally, let's connect all our REWOO components into a working multiagent system. The `create_rewoo_graph` function uses Strands' `GraphBuilder` to orchestrate our three specialized agents into a sequential pipeline.

We start by creating a `GraphBuilder` instance, then add each of our custom agents as nodes in the graph:
- The **planner** for generating structured plans
- The **worker** for executing those plans with airline tools  
- The **solver** for synthesizing final responses

Next, we define the sequential flow by adding edges that connect planner → worker → solver, ensuring data flows in the correct REWOO pattern.

We set the planner as the entry point so that user queries always start with plan generation, then automatically flow through execution and synthesis. The `builder.build()` method compiles everything into an executable graph that handles the complex orchestration, data passing, and error management between agents.

This function creates a complete REWOO system that takes a user query like "change my flight to earlier time" and automatically routes it through planning, execution, and response synthesis to deliver a final answer grounded in actual airline tool results.



In [ ]:
# Finally create the graph with the 3 agent nodes
def create_rewoo_graph():   
    builder = GraphBuilder()    
    # Add the three agents
    builder.add_node(planner_agent, "planner")
    builder.add_node(worker_agent, "worker")
    builder.add_node(solver_agent, "solver")
    
    # Sequential flow: planner -> worker -> solver
    builder.add_edge("planner", "worker")
    builder.add_edge("worker", "solver")
    
    builder.set_entry_point("planner")
    return builder.build()



## Load Dataset

Now let's load the **TauBench evaluation dataset** that contains real airline customer service scenarios. We do this so that we can test our REWOO system against standardized benchmarks and measure its performance on authentic customer queries like:

- Flight changes
- Cancellations  
- Booking modifications

This loads the **single-turn airline tasks** from TauBench, which provides us with a collection of customer queries along with their expected outcomes for evaluation purposes.

In [ ]:
output_path = os.path.join("..", "data", "tau-bench", "tau_bench", "envs", f"{domain}", "tasks_singleturn.json")
with open(output_path, "r") as file:
    tasks = json.load(file)
print(len(tasks))

## Test the REWOO System

Now let's create a test function to evaluate our orchestration system on specific TauBench questions. We do this so that we can **measure execution time**, **capture detailed results** from each agent, and **save the complete interaction flow** for analysis and debugging purposes. 

This function extracts a specific test case from our dataset, runs it through the complete REWOO pipeline, and captures both console output and detailed file logs showing how each agent (**planner**, **executor** and **solver**) processes the customer query.



In [ ]:
# Create and execute
# previous without getting metrics
def test_rewoo_graph(question_id):
    task = tasks[question_id]
    print(task)

    user_query = task["question"]    
    rewoo_graph = create_rewoo_graph()    
    start=time.time()
    
    result = rewoo_graph(user_query)    
    exec_time=time.time()-start
    print("=== REWOO Multiagent Graph Results ===")
    print(f"Graph execution time: {exec_time}")
    
    print(f"Status: {result.status}")
    print(f"Total nodes: {result.total_nodes}")
    print(f"Completed nodes: {result.completed_nodes}")
   

# Change the question id
question_id = 39 #20, #48 #43 #39
test_rewoo_graph(question_id)

##NOTE: To run other queries please rerun all the other cells

# Sample ReWOO Framework Execution

## User Query
"My user id is james_taylor_7043. I want to change my upcoming one-stop flight from LAS to IAH to a nonstop flight. My reservation ID is 1N99U6. I also want to remove my checked bag and want the agent to refund me for the same."

## ReWOO Framework Response

### Planning Phase
The **Planner Agent** creates a 5-step execution plan:
Plan 1: Retrieve the current reservation details to understand the current booking
#E1 = get_reservation_details[reservation_id="1N99U6"]

Plan 2: Verify user identity and get user profile information
#E2 = get_user_details[user_id="james_taylor_7043"]

Plan 3: Search for direct flights from LAS to IAH on the same date as the current reservation
#E3 = search_direct_flight[origin="LAS", destination="IAH", date=current_flight_date_from_E1]

Plan 4: Update the reservation with the new direct flight using cabin class and payment information from the original reservation
#E4 = update_reservation_flights[reservation_id="1N99U6", cabin=current_cabin_from_E1, flights=selected_direct_flight_from_E3, payment_id=payment_method_from_E1]

Plan 5: Transfer to human agents since removing checked bags is not supported by available tools
#E5 = transfer_to_human_agents[summary="User james_taylor_7043 with reservation 1N99U6 wants to remove checked bags and get refund. This requires human agent assistance as the system cannot remove bags, only add them."]


### Execution Phase

The **Worker Agent** executes each plan step and collects evidence:

- **E1**: Retrieved reservation showing one-stop flight (LAS→PHX→IAH) with economy cabin
- **E2**: Verified user identity and payment methods (gift card available)
- **E3**: Found 2 direct flight options (HAT175 at 17:00, HAT266 at 13:00)
- **E4**: Successfully updated reservation with both direct flight options
- **E5**: Transferred bag removal request to human agents

### Solution Phase
The **Solver Agent** synthesizes the evidence into a final response:

> "I've updated your reservation 1N99U6 from a one-stop flight to direct flights between LAS and IAH. You now have two options:
> 1. HAT175: Departing 17:00, arriving 20:00
> 2. HAT266: Departing 13:00, arriving 16:00
> 
> For removing your checked bag and processing the refund, I've transferred your request to our human agents as our system can only add bags, not remove them. A representative will contact you soon regarding the bag removal and refund."

## Key Observations

**What you're seeing in this response :**

- **Sequential Planning**: The planner breaks down the complex multi-part request into logical, dependent steps that build on each other's results

- **Evidence Collection**: Each execution step (#E1-#E5) generates structured evidence that gets passed between agents, demonstrating the framework's ability to maintain context across operations

- **Dynamic Parameter Resolution**: Notice how `current_flight_date_from_E1` and `current_cabin_from_E1` are resolved from previous steps' results, showing inter-step dependency handling

- **Tool Limitation Recognition**: The framework intelligently identifies when available tools cannot complete a task (bag removal) and routes to appropriate alternatives (human agents)

- **Comprehensive Solution**: The final response addresses both parts of the user's request - flight change (completed) and bag removal (escalated) - demonstrating the solver's ability to synthesize partial completions into coherent responses



## Congrats!

Congratulations! You've successfully created and tested a REWOO pattern implementation using Strands multiagent orchestration. This system demonstrates:

- **Three-agent architecture** with distinct Planner, Worker, and Solver roles for systematic task decomposition
- **Structured reasoning workflow** separating planning (#E1, #E2, #E3 format) from execution and synthesis
- **Strands GraphBuilder integration** using custom Agent classes with stream_async() methods for multiagent coordination
- **Evidence-based processing** where each agent builds upon structured outputs from previous agents
- **Airline domain integration** with comprehensive MAbench tool execution and TauBench evaluation datasets

The REWOO pattern excels at complex multi-step tasks requiring reliable execution, such as airline customer service scenarios where systematic planning, tool execution, and evidence-based responses ensure accuracy and consistency in customer interactions.

